# Bloco residual

### Implementação básica de um bloco residual

In [1]:
import torch
from torch import nn

class ResidualBlock(nn.Module):

    def __init__(self, channels):
        super().__init__()

        self.conv1 = nn.Conv2d(channels, channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(channels)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(channels, channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(channels)
        self.relu2 = nn.ReLU()

    def forward(self, x):

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # Caminho residual
        out += x
        out = self.relu2(out)

        return out
    
rb = ResidualBlock(channels=3)
x = torch.rand(8, 3, 28, 28)
rb(x).shape

torch.Size([8, 3, 28, 28])

### Implementação geral

A camada residual que implementamos possui uma importante limitação. Ela não permite alterar o número de canais ou o tamanho da saída. Faremos uma implementação mais geral que permite isso.

In [2]:
import torch
from torch import nn

class ResidualBlock(nn.Module):

    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        # Se o número de canais de entrada for diferente do número de canais de saída,
        # ou o tamanho espacial da entrada for diferente da saída, não será possível
        # realizar a soma "out += x" no método forward, pois out e x terão tamanhos
        # distintos. Nesse caso, a estratégia usual é criar uma camada de convolução
        # com kernel de tamanho 1x1 que ajusta o número de canais e o tamanho da saída
        # do caminho residual.
        if in_channels != out_channels or stride != 1:
            residual_adj = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels),
            )
        else:
            residual_adj = None

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu2 = nn.ReLU()
        self.residual_adj = residual_adj

    def forward(self, x):

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.residual_adj is None:
            x_adj = x
        else:
            # Ajusta o número de canais e tamanho do caminho residual
            x_adj = self.residual_adj(x)

        out += x_adj
        out = self.relu2(out)

        return out
    
rb = ResidualBlock(3, 16)
x = torch.rand(8, 3, 28, 28)
rb(x).shape

torch.Size([8, 16, 28, 28])

### ResNet

Podemos agora implementar a arquiterua [ResNet](https://openaccess.thecvf.com/content_cvpr_2016/html/He_Deep_Residual_Learning_CVPR_2016_paper.html), uma das arquiteturas mais utilizadas em CNNS. A implementação que faremos é inspirada na implementação do Pytorch.

In [3]:
class ResNet(nn.Module):

    def __init__(self, layers, num_classes, in_channels):
        super().__init__()

        # Primeira camada
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        # Pooling inicial para reduzir a resolução
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # 4 estágios da ResNet. O primeiro não envolve mudança de canal nem de resolução
        # Os demais dobram o número de canais e reduzem a resolução pela metade
        self.layer1 = self.make_layer(64, 64, layers[0])
        self.layer2 = self.make_layer(64, 128, layers[1], stride=2)
        self.layer3 = self.make_layer(128, 256, layers[2], stride=2)
        self.layer4 = self.make_layer(256, 512, layers[3], stride=2)
        # Fixa a saída em um tensor de tamanho bs x 512 x 1 x 1
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        # Camada de classificação
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, in_channels, out_channels, num_blocks, stride=1):
        layers = []
        # O primeiro bloco residual muda o número de canais e resolução
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        # Os demais blocos apenas fazem a convolução
        for i in range(1, num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        # Mesmo que x.reshape(x.shape[0], -1)
        x = x.flatten(1)
        x = self.fc(x)

        return x

def resnet18(num_classes, in_channels):
     '''O nome resnet18 significa que a rede possui 18 camadas com 
     parâmetros (17 convolucionais + 1 camada linear final).'''
     return ResNet(layers=[2, 2, 2, 2], num_classes=num_classes, in_channels=in_channels)

model = resnet18(10, 1)
x = torch.rand(8,1,28,28)
y = model(x)

Outros modelos da família ResNet:

In [4]:
def resnet34(num_classes, in_channels):
     '''34 camadas'''
     return ResNet(layers=[3, 4, 6, 3], num_classes=num_classes, in_channels=in_channels)

def resnet50(num_classes, in_channels):
     '''50 camadas'''
     return ResNet(layers=[3, 4, 6, 3], num_classes=num_classes, in_channels=in_channels)

def resnet101(num_classes, in_channels):
     '''101 camadas'''
     return ResNet(layers=[3, 4, 23, 3], num_classes=num_classes, in_channels=in_channels)

def resnet152(num_classes, in_channels):
     '''152 camadas'''
     return ResNet(layers=[3, 8, 36, 3], num_classes=num_classes, in_channels=in_channels)

Nota: A implementação real do Pytorch usa um bloco residual um pouco diferente para as resnets 50, 101 e 152, o que torna a implementação um pouco mais complexa.

### Modelos ResNet do Pytorch

O Pytorch possui a arquitetura ResNet implementada, que pode ser importada em qualquer código:

In [5]:
from torchvision import models

model = models.resnet18()
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Vamos contar quantas camadas estão presentes nos modelos do Pytorch:

In [6]:
def model_stats(model):
    '''Retorna o número de camadas e o número de parâmetros (em milhões) do modelo.'''

    layers = 0  # Número de camadas
    for name, module in model.named_modules():
        # Downsample é o nome do caminho residual usado pelo Pytorch. Removemos
        # ele para não contar duas vezes a mesma camada.
        if isinstance(module, nn.Conv2d) and 'downsample' not in name:
            layers += 1
    # Camada linear final
    layers += 1

    params = 0   # Número de parâmetros
    for param in model.parameters():
        params += param.numel()

    return layers, params/1e6

print(model_stats(models.resnet18()))
print(model_stats(models.resnet34()))
print(model_stats(models.resnet50()))
print(model_stats(models.resnet101()))
print(model_stats(models.resnet152()))

(18, 11.689512)
(34, 21.797672)
(50, 25.557032)
(101, 44.54916)
(152, 60.192808)
